<h1 style="text-align: center; font-size: 500%; text-decoration: underline;">Concluding Assignment<h1>
<h2 style="text-align: center; font-size: 250%;">Ido Israeli (ID - 212432439)<br>Jonathan Derhy (ID - 315856377)<h2>
<h2 style="text-align: center; font-size: 400%; text-decoration: underline;">Part II<h2>

<h3 style="text-decoration: underline;">Imports<h3>

In [885]:
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

<h3 style="text-decoration: underline;">Part A: Preparing the data to train a linear classifier<h3>